In [ ]:
# 使用单个nvidia gpu进行计算

In [1]:
!nvidia-smi

Mon Dec 16 16:32:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.70                 Driver Version: 537.70       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  | 00000000:01:00.0  On |                  N/A |
| 34%   33C    P5              18W / 170W |   3083MiB / 12288MiB |     20%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
# 在PyTorch中，每个数组都有一个设备（device），我们通常将其称为环境
# 默认情况下，所有变量和相关的计算都分配给CPU
# 有时环境可能是GPU

# 当我们跨多个服务器部署作业时，事情变得更加棘手
# 通过智能地将数组分配给环境，我们可以最大限度地减少在设备之间传输数据的时间
# 例如，当在带有GPU的服务器上训练神经网络时，我们通常希望模型的参数在GPU上

In [5]:
# 计算设备
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')  # CPU表示，GPU表示，有多个GPU用0/1/...表示，cuda:0和cuda等价
# CPU设备意味着所有物理CPU和内存，意味着PyTorch的计算将尝试使用所有CPU核心
# GPU设备只代表一个卡和相应的显存

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [7]:
torch.cuda.device_count()  # 查询可用GPU数量

1

In [9]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]


try_gpu(), try_gpu(10), try_all_gpus()


(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [11]:
# 张量与GPU
x = torch.tensor([1, 2, 3])
x.device  # 默认情况下，张量在CPU上创建

device(type='cpu')

In [18]:
X = torch.ones(2, 3, device=try_gpu())  # 创建张量时指定存储设备
X  # 只消耗这个GPU的显存，一般来说，我们需要确保不创建超过GPU显存限制的数据

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [17]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1725, 0.9614, 0.1143],
        [0.0307, 0.6892, 0.5907]])

In [19]:
X + Y  # 数据不在同一个设备上，会报错

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [21]:
Z = Y.cuda(0)  # 将cpu上的移到gpu0上
print(Y)
print(Z)

tensor([[0.1725, 0.9614, 0.1143],
        [0.0307, 0.6892, 0.5907]])
tensor([[0.1725, 0.9614, 0.1143],
        [0.0307, 0.6892, 0.5907]], device='cuda:0')


In [22]:
X + Z 

tensor([[1.1725, 1.9614, 1.1143],
        [1.0307, 1.6892, 1.5907]], device='cuda:0')

In [28]:
Z.cuda(0) is Z  # 调用Z.cuda(0)将返回Z，不会复制并分配新的内存

True

In [29]:
# 在设备（CPU/GPU/其他机器）之间传输数据比计算慢得多，这使得并行化变得更加困难
# 拷贝操作要格外小心。根据经验，多个小操作比一个大操作糟糕得多；一次执行几个操作比代码中散布的许多单个操作要好得多。
# 当我们print张量或者将张量转换为NumPy格式时，如果数据不在内存中，框架会首先将其复制到内存中，这会导致额外的传输开销



In [30]:
# 神经网络与GPU
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [31]:
net(X)

tensor([[-1.0892],
        [-1.0892]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [32]:
# 确认模型参数存储在同一个GPU上
net[0].weight.data.device

device(type='cuda', index=0)

In [ ]:
# 不经意地移动数据可能会显著降低性能。一个典型的错误如下：
# 计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy ndarray中）时，将触发全局解释器锁，
# 从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。